# NYC 311 Service Requests Data Pipeline

*Library + jobs to fetch, cache, and analyze NYC 311 service request data.*

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# As Jobs

## LTP Pipeline

In [ ]:
from pathlib import Path

# Per-user, cross-platform base under the home dir
# retrieve for one day only:
dir_base = Path.home() / "TASK_HBC_TSY"

%cd -q ..
!python -m hbc.jobs.dispatch \
  --job-name=job_fetch_nyc_open_data_311_service_requests \
  --as-of=20091231 \
  --dir-base={dir_base} \
  --incremental=True \
  --log-level=INFO
%cd -q notebooks/

In [ ]:
# restore cache integrity for the last missing dates
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_fetch_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --incremental=false \
      --log-level=INFO \
      --last-missing-dates=5
%cd -q notebooks/

## Analytics

In [ ]:
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_analyse_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --log-level=INFO \
      --n-worst=10 \
      --n-best=10 \
      --n-days=10
%cd -q notebooks/

***

# As Library 

### Imports

In [2]:
import sys
from pathlib import Path

p = str(Path.cwd().parent) # one dir up
if p not in sys.path:
    sys.path.insert(0, p)

import pandas as pd
import os
import numpy as np
import logging
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Api

In [3]:
from hbc import app_context, DataContainer, utils as ul

In [4]:
from hbc.quant.analysis import AnalyticalEngine
from hbc.quant.plots import PlotEngine

In [5]:
app_context

AppContext
as_of : 2025-12-24
dir_analytics: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/ANALYTICS'),
dir_base: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp'),
dir_cache: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/CACHE'),
dir_logging: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS')

## Logging

In [10]:
import logging
logger = logging.getLogger()

In [11]:
# Console-only (no file writes):
ul.conf_log(level=logging.DEBUG, console=True, file=False, reset_handlers=True)

# File-only (no console output at all):
ul.conf_log(level=logging.INFO, console=False, file=True, reset_handlers=True)

# Both:
ul.conf_log(level=logging.DEBUG, console=True, file=True, reset_handlers=True)

Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt
Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt


<RootLogger root (DEBUG)>

In [12]:
logger.debug('message')

2025-12-24 20:12:40 1324733693.py           1 DEBUG root    : message


## DataContainer

In [ ]:
dc = DataContainer('nyc_open_data_311_service_requests')

In [ ]:
# we retrieve 100 rows
dc.get()

In [ ]:
dc.get(where=f"created_date = '{ul.date_as_iso_format(ul.str_as_date('20091231'))}' ")

In [ ]:
dc.to_cache()

In [ ]:
dc.from_cache().head()

In [ ]:
', '.join(dc.all_cached_dates)

## Analytics

In [ ]:
df = dc.df

In [ ]:
cols = ul.cols_as_named_tuple(df)

In [ ]:
df = df[~df[cols.DROP_FLAG]]

In [ ]:
df["hbc_days_to_close"] = (
    pd.to_datetime(df[cols.closed_date])
    - pd.to_datetime(df[cols.created_date])
).dt.days.astype("Int64")
cols = ul.cols_as_named_tuple(df)

In [ ]:
m = df[cols.hbc_days_to_close] == 0
df_closed_not_same_day = df[~m]

In [ ]:
path = ul.path_to_str(
                ul.mk_dir(ul.get_dir_analytics() / "plots")
                / "closed_requests_by_location.html"
            )
_ = PlotEngine.plot_geo_map(
            df=df_closed_not_same_day,
            col_latitude=cols.latitude,
            col_longitude=cols.longitude,
            aggregation="count",
            round_precision=3,
            cluster=True,
            start_zoom=11,
            tiles="CartoDB positron",
            savepath= path
        )
print(path)

In [ ]:
# by agency
res = AnalyticalEngine.descriptive_stats(
    n_best=10,
    n_worst=10,
    df=df_closed_not_same_day,
    col_metric=cols.hbc_days_to_close,
    group=[
        cols.agency,
        cols.agency_name,
    ],
)


In [ ]:
res.keys()

In [ ]:
res['worst']

### new dataset: nyc_open_data_311_call_center_inquiry:

In [ ]:
dc = DataContainer('nyc_open_data_311_call_center_inquiry')

In [ ]:
dc.get()

In [ ]:
dc.get(where=f"agency='NYPD'", limit=250)

In [ ]:
dc.df.shape

In [ ]:
dc.df.head()

In [ ]:
dc.df.head()

In [ ]:
dc.get(where=f"date = '{ul.date_as_iso_format(ul.str_as_date('2010-01-03'))}'")

In [ ]:
dc.df.head()

### new_dataset: nyc_open_data_311_customer_satisfaction_survey

In [ ]:
dc = DataContainer('nyc_open_data_311_customer_satisfaction_survey')

In [ ]:
dc.get()

In [ ]:
dc.get(where=f"answer_satisfaction='Neutral'")

In [ ]:
dc.df.head()

# Cache: SQLLite DataBase**

In [64]:
import hbc

In [65]:
import os
os.environ["HBC_DB_PATH"] = "/Users/alexandershubert/git/hbc_tsy_enhanced/hbc_db/hbc.db"
os.environ["HBC_API_URL"] = "http://localhost:5047"

from hbc import DataContainer
from hbc.ltp.persistence.db import SqlLiteDataBase

db = SqlLiteDataBase()
dc = DataContainer("nyc_open_data_311_customer_satisfaction_survey")
dc.get(limit=10)

2025-12-24 20:28:10 connectionpool.py    1049 DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): data.cityofnewyork.us:443
2025-12-24 20:28:10 connectionpool.py     544 DEBUG urllib3.connectionpool: https://data.cityofnewyork.us:443 "GET /resource/kizp-4dfk.json?%24limit=10 HTTP/1.1" 200 None
2025-12-24 20:28:10 fetch_nycopen.py       71 INFO  root    : Fetched 10 rows
2025-12-24 20:28:10 base.py                52 INFO  root    : using validator: ValidatorGeneric
2025-12-24 20:28:10 base.py                53 INFO  root    : cleaning...
2025-12-24 20:28:10 base.py                56 INFO  root    : normalizing...
2025-12-24 20:28:10 base.py                59 INFO  root    : validating...
2025-12-24 20:28:10 base.py                62 INFO  root    : finalizing...


In [66]:
dc.to_cache()

2025-12-24 20:28:11 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047


   year    campaign             channel                          survey_type  \
0  2021  Campaign 5  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   
1  2021  Campaign 5  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   
2  2022  Campaign 2  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   
3  2022  Campaign 2  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   
4  2022  Campaign 4  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   
5  2023  Campaign 3  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   
6  2023  Campaign 3  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   
7  2023  Campaign 4  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   
8  2020       Pilot  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   
9  2020       Pilot  NYC311 Call Center  Customer Satisfaction (CSAT) Survey   

                start_time          completion_time  \
0  2021-12-08T15:09:24.000  2021-12-08T15:18:59.000   
1  2021-1

2025-12-24 20:28:12 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "POST /surveys HTTP/1.1" 201 None
2025-12-24 20:28:12 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 20:28:12 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "POST /surveys HTTP/1.1" 201 None
2025-12-24 20:28:12 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 20:28:12 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "POST /surveys HTTP/1.1" 201 None
2025-12-24 20:28:12 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 20:28:12 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "POST /surveys HTTP/1.1" 201 None
2025-12-24 20:28:12 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP con

In [51]:
db.update_surveys_table(dc.df)

2025-12-24 20:23:50 db.py                 184 INFO  root    : Posting 10 survey rows to http://localhost:5047/surveys/batch (verify=None)
2025-12-24 20:23:50 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 20:23:50 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "POST /surveys/batch HTTP/1.1" 200 None
2025-12-24 20:23:50 db.py                 204 INFO  root    : Synced 10 survey rows via batch API (status 200)


[200, 200, 200, 200, 200, 200, 200, 200, 200, 200]

In [56]:
dc.df.head()

,year,campaign,channel,survey_type,start_time,completion_time,survey_language,overall_satisfaction,wait_time,agent_customer_service,agent_job_knowledge,answer_satisfaction,nps
0,2021,Campaign 5,NYC311 Call Center,Customer Satisfaction (CSAT) Survey,2021-12-08T15:09:24.000,2021-12-08T15:18:59.000,English (United States)‎,Agree,Agree,Agree,Agree,Disagree,7
1,2021,Campaign 5,NYC311 Call Center,Customer Satisfaction (CSAT) Survey,2021-12-13T09:17:09.000,2021-12-13T09:19:31.000,English (United States)‎,Agree,Agree,Agree,Neutral,Neutral,7
2,2022,Campaign 2,NYC311 Call Center,Customer Satisfaction (CSAT) Survey,2022-03-17T13:12:59.000,2022-03-17T13:14:57.000,English (United States)‎,Agree,Agree,Agree,Agree,Neutral,8
3,2022,Campaign 2,NYC311 Call Center,Customer Satisfaction (CSAT) Survey,2022-03-17T13:46:58.000,2022-03-17T13:48:40.000,English (United States)‎,Strongly Agree,Agree,Agree,Agree,Agree,10
4,2022,Campaign 4,NYC311 Call Center,Customer Satisfaction (CSAT) Survey,2022-09-28T17:09:18.000,2022-09-28T17:14:02.000,English (United States)‎,Agree,Agree,Strongly Agree,Agree,Agree,9


In [59]:
db.run_query(f"SELECT * FROM {dc.moniker}").shape

(20, 14)

In [63]:
db.execute(f"DELETE FROM {dc.moniker}")

20

In [40]:
dc.df.head()


db.update_surveys_table(dc.df.head())  # now per-row POST

print(db.run_query(f"SELECT count(*) FROM {dc.moniker}"))

2025-12-24 20:21:57 connectionpool.py    1049 DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): data.cityofnewyork.us:443
2025-12-24 20:21:57 connectionpool.py     544 DEBUG urllib3.connectionpool: https://data.cityofnewyork.us:443 "GET /resource/kizp-4dfk.json?%24limit=10 HTTP/1.1" 200 None
2025-12-24 20:21:57 fetch_nycopen.py       71 INFO  root    : Fetched 10 rows
2025-12-24 20:21:57 base.py                52 INFO  root    : using validator: ValidatorGeneric
2025-12-24 20:21:57 base.py                53 INFO  root    : cleaning...
2025-12-24 20:21:57 base.py                56 INFO  root    : normalizing...
2025-12-24 20:21:57 base.py                59 INFO  root    : validating...
2025-12-24 20:21:57 base.py                62 INFO  root    : finalizing...
2025-12-24 20:21:57 db.py                 184 INFO  root    : Posting 5 survey rows to http://localhost:5047/surveys/batch (verify=None)
2025-12-24 20:21:57 connectionpool.py     241 DEBUG urllib3.connectionpool: St

   count(*)
0        20


In [43]:
db.execute(f'DELETE FROM {dc.moniker}')

20

In [13]:
import os

In [20]:
db = SqlLiteDataBase()

In [16]:
db.all_dbs

['main:/Users/alexandershubert/git/hbc_tsy_enhanced/hbc_db/hbc.db']

In [17]:
db.all_tables

['main:__EFMigrationsHistory',
 'main:nyc_open_data_311_customer_satisfaction_survey',
 'main:sqlite_sequence']

In [18]:
db.run_query(f"SELECT * from  {dc.moniker}").shape

(0, 14)

In [41]:
dc.to_cache()

db.run_query(f"SELECT * from  {dc.moniker}").shape

2025-12-24 20:09:30 db.py                 200 INFO  root    : Posting 10 survey rows to http://localhost:5047/surveys/batch (verify=False)
2025-12-24 20:09:30 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 20:09:30 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "POST /surveys/batch HTTP/1.1" 200 None
2025-12-24 20:09:30 db.py                 220 INFO  root    : Synced 10 survey rows via batch API (status 200)


(0, 14)

In [42]:
dc.to_cache()

2025-12-24 20:09:31 db.py                 200 INFO  root    : Posting 10 survey rows to http://localhost:5047/surveys/batch (verify=False)
2025-12-24 20:09:31 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 20:09:32 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "POST /surveys/batch HTTP/1.1" 200 None
2025-12-24 20:09:32 db.py                 220 INFO  root    : Synced 10 survey rows via batch API (status 200)


In [16]:
db.update_surveys_table(dc.df.head(), verify=False)

2025-12-24 19:57:31 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 19:57:31 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "POST /surveys/batch HTTP/1.1" 200 None
2025-12-24 19:57:31 db.py                 203 INFO  root    : Synced 5 survey rows via batch API (status 200)


[200, 200, 200, 200, 200]

In [20]:
db.run_query(f"SELECT * from  nyc_open_data_311_customer_satisfaction_survey")

,id,year,campaign,channel,survey_type,start_time,completion_time,survey_language,overall_satisfaction,wait_time,agent_customer_service,agent_job_knowledge,answer_satisfaction,nps


In [19]:
db.all_tables

['main:__EFMigrationsHistory',
 'main:nyc_open_data_311_customer_satisfaction_survey',
 'main:sqlite_sequence']

In [20]:
db.all_tables

['main:__EFMigrationsHistory',
 'main:nyc_open_data_311_customer_satisfaction_survey',
 'main:sqlite_sequence']

In [26]:
db.run_query(f"SELECT * from  nyc_open_data_311_customer_satisfaction_survey")

,id,year,campaign,channel,survey_type,start_time,completion_time,survey_language,overall_satisfaction,wait_time,agent_customer_service,agent_job_knowledge,answer_satisfaction,nps
0,1,2025,TEST,CLI,None,2025-01-01 00:00:00,None,None,None,None,None,None,None,None


In [42]:
import os, requests

API = os.environ.get("HBC_API_URL", "http://localhost:5047").rstrip("/")

# 1) Ping root (or swagger) to confirm the server responds
r = requests.get(f"{API}/swagger")
print("Swagger status:", r.status_code)

2025-12-24 20:22:19 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 20:22:19 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "GET /swagger HTTP/1.1" 301 0
2025-12-24 20:22:19 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "GET /swagger/index.html HTTP/1.1" 200 None


Swagger status: 200


In [24]:
# 2) List current surveys
r = requests.get(f"{API}/surveys")
print("GET /surveys:", r.status_code, "count:", len(r.json()) if r.ok else r.text)

# 3) Post a single test survey (minimal fields)
payload = {
    "year": "2025",
    "campaign": "TEST",
    "channel": "CLI",
    "start_time": "2025-01-01T00:00:00",
}
r = requests.post(f"{API}/surveys", json=payload)
print("POST /surveys:", r.status_code, r.text)

# 4) Fetch again to see if it’s stored
r = requests.get(f"{API}/surveys")
print("After insert count:", len(r.json()) if r.ok else r.text)

2025-12-24 19:59:17 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 19:59:17 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "GET /swagger HTTP/1.1" 301 0
2025-12-24 19:59:17 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "GET /swagger/index.html HTTP/1.1" 200 None
2025-12-24 19:59:17 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 19:59:17 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "GET /surveys HTTP/1.1" 200 None
2025-12-24 19:59:17 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047


Swagger status: 200
GET /surveys: 200 count: 0


2025-12-24 19:59:17 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "POST /surveys HTTP/1.1" 201 None
2025-12-24 19:59:17 connectionpool.py     241 DEBUG urllib3.connectionpool: Starting new HTTP connection (1): localhost:5047
2025-12-24 19:59:17 connectionpool.py     544 DEBUG urllib3.connectionpool: http://localhost:5047 "GET /surveys HTTP/1.1" 200 None


POST /surveys: 201 {"id":1,"year":"2025","campaign":"TEST","channel":"CLI","survey_type":null,"start_time":"2025-01-01T00:00:00","completion_time":null,"survey_language":null,"overall_satisfaction":null,"wait_time":null,"agent_customer_service":null,"agent_job_knowledge":null,"answer_satisfaction":null,"nps":null}
After insert count: 1


In [ ]:
db.db_path

In [ ]:
db.db_path

In [ ]:
dc.get(where="overall_satisfaction='Neutral'", limit=300)

In [ ]:
dc.to_cache()

In [ ]:
logger.setLevel(logging.INFO)

In [ ]:
dc.to_cache()

In [ ]:
db.update_surveys_table(dc.df.head(), verify=False)

In [ ]:
db.update_surveys_table(dc.df.head(20).tail(6), verify=False)

In [ ]:
db.run_query(f"SELECT * from  {dc.moniker}")

In [ ]:
db.db_path

In [ ]:
db.execute(f"DELETE from  {dc.moniker}")

In [ ]:
db.all_dbs

In [ ]:
db.all_tables